In [64]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import argparse
import random
import torchaudio
import pyloudnorm as pyln
import glob

n = 1
table = pq.read_table(os.path.join('..', 'data', f'out-{n}-600.parquet')).to_pandas()
samples, rate = torchaudio.load(f'../data/out-{n}-600.mp3')

def find_splits(min_frames=30, min_pause_frames=20):
    last_i = 0
    last_split = 0
    last_dur = 0
    max_i = table.shape[0]
    splits = []
    for i in range(max_i):
        if table['label'][i] == 'Neutral':
            fi = int(table['frame'][i])
            fi2 = int(table['frame'][i + 1]) if i < max_i - 1 else 0
            dur = fi2 - fi
            if i > 0 and i < max_i - 1 and dur > min_pause_frames and fi - last_split >= min_frames:
                splits.append([last_split + last_dur // 2, fi + dur // 2])
                data = table[last_split:i]
                last_i = i
                last_split = fi
                last_dur = fi2 - fi
    return splits

def extract(start, end):
    pass


In [66]:
find_splits(81, 189)

[[0, 19311]]